Libraries

In [16]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import os

fetch page content

In [6]:
# headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
}

# fetch the page content
url_root = "https://www.immoweb.be/en/search/for-sale"
endpoint = '?countries=BE&page='
apikey = '&orderBy=relevance'
url = url_root+endpoint+apikey

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
<head>
<meta charset="utf-8"/>
<script>
            if (!Array.prototype.flat) { window.location.replace('https://www.immoweb.be/en/outdated-browser') }
        </script>
<script type="text/javascript">
    const variantName = "AI_in_Advanced_Search";
    const isABTestWithCookie = false;

    /* START: AB Test Homepage - AI tab for unauthenticated users. */
    const LDConsent = hasCookieConsentById("vagdYWLSXoC06v");
    /* END: AB Test Homepage - AI tab for unauthenticated users. */

    if (
        /* START: AB Test Homepage - AI tab for unauthenticated users. */
        !LDConsent ||
        /* END: AB Test Homepage - AI tab for unauthenticated users. */
        !variantName || isABTestWithCookie) { // AB test is not configured or started using cookie for storing variant
        localStorage.removeItem("ab-test");
        window.ABTestVariant = null;
    } else {
        setABTestVariant();
    }


getting houses urls

In [19]:
current_dir = os.getcwd()
driver_path  = os.path.join(current_dir, "chromedriver")

driver = webdriver.Chrome(service=Service(driver_path))


houses_url = []

for i in range(1, 4):  
    # Construct the URL for each page
    url = f'https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={i}&orderBy=relevance'
    driver.get(url)

    wait = WebDriverWait(driver, 30)  
    try:        
        house_links = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/en/classified')]")))
        print(f"Found {len(house_links)} house links on page {i}.")
        
        for link in house_links:
            href = link.get_attribute('href')
            if href:  
                houses_url.append(href)

    except Exception as e:
        print(f"Error on page {i}: {e}")
        continue

    # Pause to avoid overloading the server
    time.sleep(2)  

# Save the list of URLs to a CSV file
with open('houses_urls.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["House URL"])  
    for url in houses_url:
        writer.writerow([url])  

# Close the WebDriver after scraping is done
driver.quit()

print(f"Scraping complete. {len(houses_url)} house URLs saved to 'houses_urls.csv'.")

Found 60 house links on page 1.
Found 60 house links on page 2.
Found 61 house links on page 3.
Scraping complete. 181 house URLs saved to 'houses_urls.csv'.


Getting the houses info from urls

In [ ]:
# This is for Overview extracting

# locality = listing.find('span', class_='overview__text').text.strip()
# print(locality)
# property_type = "Apartment"
# subtype = listing.find('span', class_='overview__text').text.strip()
# #clean price
# price = listing.find('span', class_='overview__text').text.strip().replace('€', '').replace('.', '').replace(',', '').strip()
# type_of_sale = "For Sale"
# rooms = listing.find('span', class_='overview__text').text.strip()
# living_area = listing.find('span', class_='overview__text').text.strip().replace('m²', '').strip()
# kitchen = listing.find('span', class_='overview__text').text.strip()
# furnished = listing.find('span', class_='overview__text').text.strip()
# open_fire = listing.find('span', class_='overview__text').text.strip()
# terrace = listing.find('span', class_='overview__text').text.strip()
# garden = listing.find('span', class_='overview__text').text.strip()
# surface_land = listing.find('span', class_='overview__text').text.strip()
# surface_area_plot_land =listing.find('span', class_='overview__text').text.strip()
# number_of_facades =listing.find('span', class_='overview__text').text.strip()
# swimming_pool = listing.find('span', class_='overview__text').text.strip()
# state_of_building = listing.find('span', class_='overview__text').text.strip()

excel storage

In [ ]:
import pandas as pd

file_path = '/home/betty/Becode_training_path/Projects/Immo-Eliza-project/zipcodes_prov_fr.xlsx'
file_path2 = '/home/betty/Becode_training_path/Projects/Immo-Eliza-project/zipcodes_prov_nl.xlsx'
df = pd.read_excel(file_path2)
print(df.head())


   Postcode                                         Instelling
0       612                                        Sinterklaas
1      1005   Verenigde Vergadering van de Gemeenschappelijke 
2      1006          Raad van de Vlaamse Gemeenschapscommissie
3      1007  Assemblée de la Commission Communautaire Franç...
4      1008                  Kamer van Volksvertegenwoordigers


In [12]:
import pandas as pd
file_path_fr = '/home/betty/Becode_training_path/Projects/Immo-Eliza-project/zipcodes_prov_fr.xlsx'
file_path_nl = '/home/betty/Becode_training_path/Projects/Immo-Eliza-project/zipcodes_prov_nl.xlsx'

df_fr = pd.read_excel(file_path_fr, engine='openpyxl')
df_nl = pd.read_excel(file_path_nl, engine='openpyxl')


dict_fr = pd.Series(df_fr['Code postal'].values, index=df_fr['Institution']).to_dict()
dict_nl = pd.Series(df_nl['Postcode'].values, index=df_nl['Instelling']).to_dict()

# Combine both dictionaries
combined_dict = {**dict_fr, **dict_nl}

# Print the combined dictionary
print("\nCombined Dictionary:")
print(combined_dict)



Combined Dictionary:
{'Saint-Nicolas': 612, 'Assemblée Réunie de la Commission Communautaire': 1005, 'Raad van de Vlaamse Gemeenschapscommissie ': 1006, 'Assemblée de la Commission Communautaire Française': 1007, 'Chambre des Représentants': 1008, 'Senat de Belgique': 1009, 'Vlaams parlement': 1011, 'Parlement de la Communauté française': 1012, 'Organisations Sociales Chrétiennes': 1031, 'RTL-TVI': 1033, 'Ministère de la Région de Bruxelles Capitale': 1035, 'International Press Center': 1041, 'VRT': 1043, 'RTBF': 1044, 'European External Action Service': 1046, 'Parlement Européen': 1047, 'Union Européenne - Conseil': 1048, 'Union Européenne - Commission': 1049, 'Bruxelles X': 1099, 'Postcheque': 1100, 'Scanning': 1101, 'ACTISOC': 1105, 'OTAN': 1110, 'SPF Mobilité': 1212, 'HighCo DATA': 1733, 'Cargovil': 1804, 'VTM': 1818, 'Brucargo': 1931, 'Office Exchange Brussels Airport Remailing': 1934, 'Corporate Village': 1935, 'Antwerpen x': 2099, 'Liège X': 4099, 'SA SudPresse': 5010, 'Parleme